In [1]:
# Loading packages
library(data.table)
library(nloptr)
library(ggplot2)
library(reshape2)
library(ggplot2)
library(dplyr)
library(Hmisc)
library(tidyr)
library(readr)
library(dichromat)


Warning message:
"package 'data.table' was built under R version 3.6.3"Warning message:
"package 'nloptr' was built under R version 3.6.3"
Attaching package: 'reshape2'

The following objects are masked from 'package:data.table':

    dcast, melt

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'Hmisc' was built under R version 3.6.3"Loading required package: lattice
Loading required package: survival
Warning message:
"package 'survival' was built under R version 3.6.3"Loading required package: Formula
Warning message:
"package 'Formula' was built under R version 3.6.3"
Attaching package: 'Hmisc'

The following objects are masked from 'package:dplyr':

    src, s

In [2]:
# Human played tasks
datadir <- setwd("C:/Users/selin/OneDrive/Documents/GitHub/SelinProject_SVV")
hpTasks <- read.csv('ModelInputData.csv')
hpTasks <- hpTasks[,-1]
hpTasks$outcomeIsLose <- 1 - hpTasks$outcome
hpTasks$outcome <- NULL # Remove "outcome" now that we have "outcomeIsLose"
names(hpTasks)[names(hpTasks) == "reward"] <- "magnitudeLost"
head(hpTasks)

subject,trialNumber,chosenStimulus,magnitudeStimulus1,magnitudeStimulus2,magnitudeLost,blockSocial,blockStable,outcomeIsLose
53,1,2,40,20,20,1,0,1
53,2,1,10,10,0,1,0,0
53,3,1,10,80,10,1,0,1
53,4,2,10,60,60,1,0,1
53,5,1,20,60,20,1,0,1
53,6,1,10,20,0,1,0,0


In [3]:
# We have different versions of calculating the expected utility we can use and compare results.
# Only the multiplicative model uses expected utilities.

smooth_expected_utility <- function(eta, P_lose, magnitude) {
    # Probabilities are between 0 and 1. In this domain this asymptote goes from
    # infinity to 0 and goes through 1 at 0.5. Raising it to a higher power makes it steeper.
    asymptote <- (1 - P_lose) / P_lose
    return (-magnitude / (1 + asymptote ^ eta))
}

# Define which variant we'll be using in the model
expected_utility <- smooth_expected_utility

In [4]:

update_P_lose <- function(alpha, P_lose_trial, chosenStimulus, outcomeIsLose, updateRule) {
    # A player might not have made a choice, in which case there is no update
    if (is.na(chosenStimulus) | chosenStimulus < 1) return (P_lose_trial)
    PPE <- outcomeIsLose - P_lose_trial[chosenStimulus] 
    P_lose_trial[chosenStimulus] <- P_lose_trial[chosenStimulus] + alpha * PPE 
    if (updateRule == "independent" | updateRule == "single"){
        P_lose_trial[3 - chosenStimulus] <- P_lose_trial[3 - chosenStimulus]
    } else if (updateRule == "dependent" | updateRule == "double"){
        P_lose_trial[3 - chosenStimulus] <- 1 - P_lose_trial[chosenStimulus]
    } else {
        print("updateRule has not been defined (indicate: independent/single OR dependent/double)")
    }
    return (P_lose_trial)
}


In [5]:
# Create a table of p_lose across the whole block
estimate_p_lose <- function(played_block, alpha, updateRule) {
    model_values <- data.frame(P_lose1 = rep(NaN, nrow(played_block)), P_lose2 = rep(NaN, nrow(played_block)))
    if (nrow(model_values) == 0) return (list(block = played_block, model_values = model_values))

    P_lose <- c(0.5, 0.5)
    for (t in 1:(nrow(played_block))) {
        model_values$P_lose1[t] <- P_lose[1]
        model_values$P_lose2[t] <- P_lose[2]
        P_lose <- update_P_lose(alpha, P_lose, played_block$chosenStimulus[t], played_block$outcomeIsLose[t], updateRule)
    }
    return(model_values)
}


# Given p_choose1 we can derive p_choose2. We add an extra column p_choice that selects 
# p_choose1 or p_choose2 per trial depending on the actual choice for that trial.
complete_p_choose <- function(played_block) {
    # Choose 2 is special because we have a forced choice, so not choosing 1 means choosing 2.
    # The model does not take the option into account of not choosing so it does not estimate the probability for that.
    # We could also compute P_choose2 as 1 / (1 + exp(-beta * (model_values$EU2 - model_values$EU1)))
    played_block$P_choose2 = 1 - played_block$P_choose1
    
    played_block$P_choice <- NaN
    
    selection = !is.na(played_block$chosenStimulus) & played_block$chosenStimulus == 1
    played_block$P_choice[selection] <- played_block$P_choose1[selection]
    
    selection = !is.na(played_block$chosenStimulus) & played_block$chosenStimulus == 2
    played_block$P_choice[selection] <- played_block$P_choose2[selection]
    
    return(played_block)
}

# Input: the data for a single block and an explanation function
# Output: the negative log likelihood (NLL) of the recorded choices made in the block given the free parameters
compute_blockNLL <- function(played_block, block_explain_function)
{
    if (nrow(played_block) == 0) {
        print("Requested NLL for empty block")
        return (0)
    }
    explained_block <- block_explain_function(played_block)$block
    NLL <- -sum(log(na.omit(explained_block$P_choice)))
    return(NLL)
}

# Input: a subject nr, the data for the entire task and an explanation function
# Output: the negative log likelihood for all the recorded choices the subject made given the free parameters
compute_subjectNLL <- function(subject, played_tasks, alphas, block_explain_function)
{
    task <- played_tasks[played_tasks$subject == subject,]
    return (
        compute_blockNLL(task[task$blockSocial == 0 & task$blockStable == 0,], function(block) block_explain_function(block, alphas[1])) +
        compute_blockNLL(task[task$blockSocial == 0 & task$blockStable == 1,], function(block) block_explain_function(block, alphas[2])) +
        compute_blockNLL(task[task$blockSocial == 1 & task$blockStable == 0,], function(block) block_explain_function(block, alphas[3])) +
        compute_blockNLL(task[task$blockSocial == 1 & task$blockStable == 1,], function(block) block_explain_function(block, alphas[4]))
    )
}


In [6]:
# This cell has functions that are different for the multiplicative and additive models

multiplicative_selector <- function(expectedUtility1, expectedUtility2, beta) {
    return (1 / (1 + exp(-beta * (expectedUtility1 - expectedUtility2)))) # B & R Formula (4)
}

additive_selector <- function(P_lose1, P_lose2, mag1, mag2, beta, phi) {
    # These differences should be positive if they favour choosing 1. Since we're dealing with losses
    # we should favour 1 if p(lose)1 < p(lose)2 and if magnitude1 < magnitude2
    prob_diff = P_lose2 - P_lose1
    reward_diff = (mag2 - mag1) / 80
    return (1 / (1 + exp(-beta * (phi * prob_diff + (1 - phi) * reward_diff)))) # B & R Formula (5)
}

# Input: the played data for a single block and free parameters
# Output: a list with two entries:
#  - the explained block
#  - intermediate model values that can be use to debug and understand how the model works

explain_block_multiplicative <- function(played_block, alpha, beta, eta, updateRule)
{
    model_values <- estimate_p_lose(played_block, alpha, updateRule)
    model_values$EU1 = expected_utility(eta, model_values$P_lose1, played_block$magnitudeStimulus1)
    model_values$EU2 = expected_utility(eta, model_values$P_lose2, played_block$magnitudeStimulus2)
    
    played_block$P_choose1 = multiplicative_selector(model_values$EU1, model_values$EU2, beta)
    played_block <- complete_p_choose(played_block)
    
    return(list(block = played_block, model_values = model_values))
}



# The same function as above but using the independent updating of probabilities for two options
explain_block_additive <- function(played_block, alpha, beta, phi, updateRule)
{
    model_values <- estimate_p_lose(played_block, alpha, updateRule)
    
    played_block$P_choose1 = additive_selector(model_values$P_lose1, model_values$P_lose2, played_block$magnitudeStimulus1, played_block$magnitudeStimulus2, beta, phi)
    played_block <- complete_p_choose(played_block)
    
    return(list(block = played_block, model_values = model_values))
}




In [52]:
played_block <- hpTasks[hpTasks$subject == 1 & hpTasks$blockSocial == 0 & hpTasks$blockStable == 0,]
result <- explain_block_additive(played_block, alpha = 0.1, beta = 0.1, phi = 0.9)
head(cbind(result$block, result$model_values))

,subject,trialNumber,chosenStimulus,magnitudeStimulus1,magnitudeStimulus2,magnitudeLost,blockSocial,blockStable,outcomeIsLose,P_choose1,P_choose2,P_choice,P_lose1,P_lose2
39721,1,121,2,60,10,10,0,0,1,0.4984375,0.5015625,0.5015625,0.50,0.50000
39722,1,122,2,40,10,10,0,0,1,0.5001875,0.4998125,0.4998125,0.50,0.55000
39723,1,123,1,10,20,0,0,0,0,0.5024500,0.4975500,0.5024500,0.50,0.59500
39724,1,124,2,20,20,20,0,0,1,0.5032625,0.4967375,0.4967375,0.45,0.59500
39725,1,125,2,60,20,20,0,0,1,0.5029237,0.4970763,0.4970763,0.45,0.63550
39726,1,126,2,20,10,10,0,0,1,0.5046812,0.4953188,0.4953188,0.45,0.67195


In [7]:
# Independent updating version
multiplicative_fitter <- list(
    #                alphas      beta, eta
    upperbound  = c(1, 1, 1, 1,   50,  10),
    lowerbound  = c(0, 0, 0, 0,    0,   0),
    name = "multiplicative",
    evaluation_function = function(x, played_tasks, subject, updateRule) {
        compute_subjectNLL(subject, played_tasks, x[1:4], function(block, a) {
            explain_block_multiplicative(block, alpha = a, beta = x[5], eta = x[6], updateRule)
        })
    }
)

# Independent updating version
additive_fitter <- list(
    #                   alpha    beta, phi
    upperbound  = c(1, 1, 1, 1,   50,   1),
    lowerbound  = c(0, 0, 0, 0,    0,   0),
    name = "additive",
    evaluation_function = function(x, played_tasks, subject, updateRule) {
        compute_subjectNLL(subject, played_tasks, x[1:4], function(block, a) {
            explain_block_additive(block, alpha = a, beta = x[5], phi = x[6], updateRule)
        })
    }
)

# This function will set up nlopt and perform parameter fitting for a single subject
fit_subject <- function(subject, played_tasks, fitter, num_init, updateRule) {
    if (!is.element(subject, played_tasks$subject)) {
        print(paste(subject, "does not appear in the given tasks"))
        print(unique(played_tasks$subject))
        return (NaN)
    }
    bestfit <- NaN
    bestNLL <- Inf
    evaluate = function(x) fitter$evaluation_function(x, played_tasks, subject, updateRule)
    updateRule = paste(updateRule)
    modelName = paste(fitter$name)
    for (initializations in 1:num_init) {
        cat("Fitting subject", subject, "init", initializations, "of", num_init, "  \r") # Fitting can take a long time so print any progress
        flush.console() # don't wait until the work is done, print it now!
        startSearch =  runif(length(fitter$upperbound), fitter$lowerbound, fitter$upperbound)
        local = list(algorithm = "NLOPT_LN_BOBYQA", maxeval = 5000, ftol_abs = 0.0001)
        options = list(algorithm = "NLOPT_GN_MLSL", ftol_abs = 0.0001, local_opts = local)
        fit_result <- nloptr(x0 = startSearch, eval_f = evaluate, lb = fitter$lowerbound, ub = fitter$upperbound, opts = options)
        if (initializations == 1 | fit_result$objective < bestNLL) {
            bestfit <- fit_result
            bestNLL <- fit_result$objective
        }
    }
    bestfit$modelName = modelName
    bestfit$updateRule = updateRule
    return (bestfit)
}

pretty_print <- function(result) {
    cat(result$message)
    cat(paste("\nIterations:", result$iterations))
    cat(paste("\nNLL:", result$objective))
    cat(paste("\nparameter:", result$solution))
    cat(paste("\nupdateRule:", result$updateRule))
    cat(paste("\nmodelName:", result$modelName))
}

In [8]:
result <- fit_subject(1, hpTasks, multiplicative_fitter, 100, "independent")
pretty_print(result)


NLOPT_MAXEVAL_REACHED: Optimization stopped because maxeval (above) was reached.
Iterations: 100
NLL: 33.1416707982393
parameter: 0.0189919953818001 
parameter: 0.22175260916318 
parameter: 0.407986452876373 
parameter: 0.334194618366132 
parameter: 16.1219745410271 
parameter: 0.000506894725504281
updateRule: independent
modelName: multiplicative